In [1]:
import pandas as pd
import numpy as np

In [3]:
raw_data_csv = pd.read_csv('Absenteeism_data.csv')

In [5]:
raw_data_csv.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [11]:
# we have no use for ID as it only identfies an employee uniquely and does not contribute to the absenteeism
# also making a checkpoint

In [39]:
df = raw_data_csv.copy()
df = df.drop(['ID'],axis=1)

### Preprocessing Reason for absence
- creating dummy variables for it as it is categorical nominal 
- and categorizing them into 4 reason types

In [42]:
df['Reason for Absence'].unique()

array([26,  0, 23,  7, 22, 19,  1, 11, 14, 21, 10, 13, 28, 18, 25, 24,  6,
       27, 17,  8, 12,  5,  9, 15,  4,  3,  2, 16], dtype=int64)

In [44]:
df['Reason for Absence'].value_counts()

Reason for Absence
23    147
28    110
27     66
13     52
0      38
19     36
22     32
26     31
25     29
11     24
10     22
18     21
14     18
1      16
7      13
12      8
21      6
6       6
8       5
9       4
5       3
16      3
24      3
15      2
4       2
3       1
2       1
17      1
Name: count, dtype: int64

In [57]:
sum(df['Reason for Absence'].value_counts())

700

In [59]:
df['Reason for Absence'].min()

0

In [61]:
df['Reason for Absence'].max()

28

In [63]:
sorted(df['Reason for Absence'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

In [48]:
# there are no null values in reasons for absence so we can proceed ahead with dummy variables

In [67]:
reason_columns = pd.get_dummies(df['Reason for Absence'],drop_first=True).astype(int)

In [73]:
reason_columns['check'] = reason_columns.sum(axis=1)

In [83]:
reason_columns['check'].value_counts()

check
1    662
0     38
Name: count, dtype: int64

In [85]:
reason_columns = reason_columns.drop(['check'], axis=1)

### Grouping the reasons into type_1, type_2, and so on to type_4.

In [98]:
reason_columns.columns.values

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 22, 23, 24, 25, 26, 27, 28], dtype=object)

In [137]:
df.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [112]:
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

In [116]:
# removing reason for absence from df and concatenating new reason_types to df

In [122]:
df = df.drop(['Reason for Absence'], axis=1)

In [133]:
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_3], axis=1)

In [139]:
new_column_values = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'reason_type_1', 'reason_type_2', 'reason_type_3', 'reason_type_4']

In [145]:
df.columns = new_column_values

In [149]:
new_column_values_reordered = ['reason_type_1', 'reason_type_2', 'reason_type_3', 'reason_type_4', 
                               'Date', 'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
                                'Children', 'Pets', 'Absenteeism Time in Hours']

In [156]:
df = df[new_column_values_reordered]

In [165]:
# checkpoint after preprocessing reasons for absence
df_reasons_processed = df.copy()

### Wroking with date to get 2 new columns 
- month value
- day of the week

In [210]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
(df['Date'][0])

Timestamp('2015-07-07 00:00:00')

In [224]:
df['Month Value'] = [x.month for x in df['Date']]
df['Day of the Week'] = df['Date'].apply(lambda x:x.dayofweek)

In [242]:
df.columns.values 

array(['reason_type_1', 'reason_type_2', 'reason_type_3', 'reason_type_4',
       'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Month Value',
       'Day of the Week'], dtype=object)

In [244]:
#  removing date and reordering month value and day of the week

In [246]:
df = df.drop(['Date'], axis =1)

In [248]:
column_reordered_after_date = ['reason_type_1', 'reason_type_2', 'reason_type_3', 'reason_type_4', 'Month Value',
       'Day of the Week',
        'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [250]:
df = df[column_reordered_after_date]

In [252]:
df

,reason_type_1,reason_type_2,reason_type_3,reason_type_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,0,7,1,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,1,1,0,0
2,0,0,0,0,7,2,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,1,2,0,4
4,0,0,0,0,7,3,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,2,2,0,8
696,1,0,0,0,5,2,225,26,28,237.656,24,1,1,2,3
697,1,0,0,0,5,3,330,16,28,237.656,25,2,0,0,8
698,0,0,0,0,5,3,235,16,32,237.656,25,3,0,0,2


In [ ]:
# transportation expense , ditance to work , age, Daily Work Load Average,Body Mass Index 
# does not have to be preprocessed as there are not missing values inthem also there are all numerical data.

#### checkpoint after date processing

In [278]:
df_date_processed = df.copy()

### preprocessing education as it is also a categorical data
- the legend is such that
- 1--> high school graduate
- 2--> graduate
- 3--> postgraduate
- 4--> a master or a doctor

In [255]:
df

,reason_type_1,reason_type_2,reason_type_3,reason_type_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,0,7,1,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,1,1,0,0
2,0,0,0,0,7,2,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,1,2,0,4
4,0,0,0,0,7,3,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,2,2,0,8
696,1,0,0,0,5,2,225,26,28,237.656,24,1,1,2,3
697,1,0,0,0,5,3,330,16,28,237.656,25,2,0,0,8
698,0,0,0,0,5,3,235,16,32,237.656,25,3,0,0,2


In [261]:
df['Education'].value_counts()

Education
1    583
3     73
2     40
4      4
Name: count, dtype: int64

In [263]:
# lets divide the numbers as 0 --> high school
# 1--> education more than high school

In [292]:
df['Education'] = df['Education'].map({
    1: 0, 
    2: 1,
    3: 1,
    4:1
})

### checkpoint after education processing

In [298]:
df_education_processed = df.copy()

In [300]:
# children and pets are not categorical but numerical, hence they are left as they a without processing.

In [302]:
df_preprocessed = df.copy()

In [304]:
df_preprocessed.head()

,reason_type_1,reason_type_2,reason_type_3,reason_type_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,0,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,0,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,0,7,3,289,36,33,239.554,30,0,2,1,2


In [306]:
# lets export this as absenteeism preprocessed.csv

In [308]:
df_preprocessed.to_csv('absenteeism_preprocessed.csv', index=False)